# quick_pp

In [ ]:
import sys
sys.path.append('..')
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from quick_pp.database.objects import Project
from quick_pp.database.db_connector import DBConnector

db_conn = DBConnector()

project_name = "MOCK_carbonate"
with db_conn.get_session() as db_session:
    # Load well from saved file
    project = Project(db_session, name=project_name)
    print(project.get_well_names())
    df = project.get_all_data()

# Batch Interpretation

In [ ]:
import pandas as pd
import numpy as np
import pickle
import json
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer

from quick_pp.lithology.carbonate import Carbonate
from quick_pp.porosity import neu_den_xplot_poro, density_porosity, rho_matrix, normalize_volumetric
from quick_pp.qaqc import badhole_flagging, handle_outer_limit, neu_den_xplot_hc_correction
from quick_pp.saturation import estimate_temperature_gradient, estimate_rw_temperature_salinity, archie_saturation
from quick_pp.ressum import calc_reservoir_summary
from quick_pp.rock_type import estimate_vsh_gr, calc_r35_perm, rock_typing, calc_fzi_perm
from quick_pp.core_calibration import sw_shf_leverett_j, sw_skelt_harrison

imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')

# Load FZI ROCK_FLAG
with open(rf'data\04_project\{project_name}\outputs\fzi_rt_model.qppm', 'rb') as file:
    fzi_rt_model = pickle.load(file)

# Load FZI model
with open(rf'data\04_project\{project_name}\outputs\fzi_model.qppm', 'rb') as file:
    fzi_model = pickle.load(file)

# Load FZI cutoffs
with open(rf'data\04_project\{project_name}\outputs\fzi_cutoffs.json', 'rb') as file:
    fzi_cutoffs = json.load(file)

# Load J parameters
with open(rf'data\04_project\{project_name}\outputs\rt_j_params.json', 'rb') as file:
    rt_j_params = json.load(file)

In [ ]:
WATER_CONTACT = {
    'HW-3': 8559,
    'HW-4': 8559,
    'HW-5': 8559,
    'HW-6': 8559,
    'HW-7': 8545,
    'HW-8': 8545,
    'HW-9': 8567,
    'HW-10': 8567,
    'HW-24': 8559,
    'HW-25': 8559,
    'HW-26': 8559,
    'HW-27': 8567,
    'HW-28': 8567,
    'HW-29': 8567,
    'HW-30': 8567,
    'HW-31': 8545,
    'HW-32': 8545,
}

In [ ]:
carbonate_type =  'limestone'  # 'dolostone'  #
model = 'single'  # 'double'  #
method = 'neu_den'  # 'pef_den'  #

for well_name, well_data in df.groupby('WELL_NAME'):
    # Load well object
    well = project.get_well(well_name)
    well_data['CPORE'] = well_data['CORE_POR'] / 100
    well_data['CPERM'] = well_data['CORE_PERM']

    # Clean up data
    well_data = badhole_flagging(well_data)
    # well_data = handle_outer_limit(well_data, fill=True)

    # Initialize lithology model
    args = {
        'litho_model': 'carb',
        'dry_calc_point': (.0, 2.71),
        'dry_clay_point': (.3, 2.7),
        'sw_water_salinity': 160e3,
        'sw_m': 2,
        'sw_n': 2,
        'hc_corr_angle': 30,
        'hc_buffer': 0.01,
        'ressum_cutoffs': dict(
            VSHALE=.5,
            PHIT=0,
            SWT=1
        ),
    }
    carb_model = Carbonate(**args)
    vclw_un, vcalc_un, vdolo_un = carb_model.estimate_lithology(
        well_data['NPHI'], well_data['RHOB'],
        model=model, method=method, carbonate_type=carbonate_type
    )
    args.update(carb_model.__dict__)
    well.update_config(args)  # Save lithology model to well

    # Set skip HC correction or not
    skip_hc_correction = False
    if skip_hc_correction:
        nphihc, rhobhc = well_data['NPHI'], well_data['RHOB']
    else:
        # Implement hydrocarbon correction
        nphihc, rhobhc, hc_flag = neu_den_xplot_hc_correction(
            well_data['NPHI'], well_data['RHOB'],
            dry_min1_point=args['dry_calc_point'],
            dry_clay_point=args['dry_clay_point'],
            corr_angle=args['hc_corr_angle'], buffer=args['hc_buffer']
        )

        # Estimate lithology
        carb_model = Carbonate(**args)
        vclw_un, vcalc_un, vdolo_un = carb_model.estimate_lithology(
            nphihc, rhobhc, model=model, method=method, carbonate_type=carbonate_type
        )

    # Estimate porosity
    phit = neu_den_xplot_poro(
        nphihc, rhobhc, model='carb',
        dry_min1_point=args['dry_calc_point'],
        dry_clay_point=args['dry_clay_point'],
    )

    # PHID needs unnormalized lithology
    rho_ma = rho_matrix(vclay=vclw_un, vcalc=vcalc_un, vdolo=vdolo_un)
    phid = density_porosity(rhobhc, rho_ma)
    
    # Fill missing values in phit with phid
    phit = np.where(np.isnan(phit), phid, phit)
    
    # Normalize lithology
    volumes = dict(
        vclay=vclw_un, vcalc=vcalc_un, vdolo=vdolo_un
    )
    volumes = normalize_volumetric(phit, **volumes)
    vclay, vcalc, vdolo = volumes['vclay'], volumes['vcalc'], volumes['vdolo']
    
    well_data['LOG_RT'] = np.log10(well_data['RT'])
    well_data['NDI'] = (2.95 - well_data['RHOB']) / 1.95
    well_data['GRN'] = MinMaxScaler().fit_transform(well_data[['GR']])
    input_features = ['GR', 'NPHI', 'RHOB', 'NDI', 'RT', 'LOG_RT', 'GRN']
    
    
    # Predict rock type
    rock_flag_ml = fzi_rt_model.predict(well_data[input_features])
    
    # Predict permeability
    temp_df = well_data.copy()
    temp_df['ROCK_PRED'] = rock_flag_ml
    input_features = ['GR', 'NPHI', 'RHOB', 'NDI', 'RT', 'LOG_RT', 'GRN', 'ROCK_PRED']
    temp_df[input_features] = imp_mean.fit_transform(temp_df[input_features])
    fzi_ml = 10**(fzi_model.predict(temp_df[input_features]))
    perm_ml = calc_fzi_perm(fzi_ml, phit)
    perm_ml = np.where(perm_ml < 0, 0, perm_ml)

    # Update rock type
    rock_flag_ml = rock_typing(fzi_ml, cut_offs=fzi_cutoffs)

    # Estimate water saturation
    temp_grad = estimate_temperature_gradient(well_data['DEPTH'], 'metric')  # Arbitrarily use MD instead of TVD
    water_salinity = args['sw_water_salinity']
    rw = estimate_rw_temperature_salinity(temp_grad, water_salinity)
    m = args['sw_m']
    swt = archie_saturation(well_data['RT'], rw, phit, m=m)
    swt = swt.clip(0, 1)
    
    # Estimate SHF
    ift = 32
    theta = 30
    ghc = .837
    gw = 1.135

    fwl = WATER_CONTACT[well_name]
    a = pd.Series(rock_flag_ml).map(lambda x: rt_j_params[str(float(x))]['a'])
    b = pd.Series(rock_flag_ml).map(lambda x: rt_j_params[str(float(x))]['b'])
    shf = sw_shf_leverett_j(
        perm_ml, phit, well_data['DEPTH'], gw=gw, ghc=ghc,
        fwl=fwl, ift=ift, theta=theta, a=a, b=b
    )

    # Estimate reservoir summary
    well_data['ZONES'] = 'ALL'
    ressum_df = calc_reservoir_summary(well_data.DEPTH, vclay, phit, swt, perm_ml.clip(.01, 1e6),
                                       zones=well_data['ZONES'], cutoffs=args['ressum_cutoffs'])

    # Update data in the project
    well_data['NPHI_HC'] = nphihc
    well_data['RHOB_HC'] = rhobhc
    # well_data['HC_FLAG'] = hc_flag
    well_data['PHIT'] = phit
    well_data['PERM'] = perm_ml.clip(.01, 1e6)
    well_data['VCALC'] = vcalc
    well_data['VDOLO'] = vdolo
    well_data['VCLAY'] = vclay
    well_data['PHID'] = phid
    well_data['RW'] = rw
    well_data['SWT'] = swt
    well_data['SHF'] = shf.clip(0, 1)
    well_data['BVW'] = phit * swt
    well_data['BVO'] = phit * (1 - swt)
    well_data['BVOH'] = phit * (1 - shf)
    well_data['VHC'] = phit * (1 - swt)
    well_data['ROCK_FLAG'] = rock_flag_ml
    well_data['ROCK_FLAG'] = well_data['ROCK_FLAG'].astype(int)

    # Save the results to the well
    well.update_data(well_data)
    well.update_ressum(ressum_df)
    project.save_well(well)
    # break


In [ ]:
from quick_pp.plotter.plotter import plotly_log
import os

df = project.get_all_data()
df.drop(columns='PHIE', inplace=True, errors='ignore')
folder = rf"outputs\{project_name}\interpreted_logs"
os.makedirs(folder, exist_ok=True)
for well, well_data in df.groupby("WELL_NAME"):
    # Plot the results
    fig = plotly_log(well_data, well_name=well, depth_uom='ft')
    # fig.show(config=dict(scrollZoom=True))
    fig.write_html(rf"{folder}\{well}_log.html", config=dict(scrollZoom=True))
    # fig.write_image(rf"{folder}\{well}_log.png")
    # break


# QC the results

In [ ]:
df = project.get_all_data()

In [ ]:
from quick_pp.qaqc import quick_compare
from quick_pp.utils import zone_flagging

df['VSHALE'] = df['VCLAY']
# df = zone_flagging(df)
compare_df, fig = quick_compare(df, return_fig=True)
compare_df = compare_df[compare_df['FLAG'] == 'all']

In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(nrows=len(df['WELL_NAME'].unique()), ncols=1, figsize=(20, len(df['WELL_NAME'].unique())), sharex=True)

df['BVO'] = df['PHIT'] * (1 - df['SWT'])
df['BVOH'] = df['PHIT'] * (1 - df['SHF'])
for ax, (well, well_data) in zip(axes, df.groupby("WELL_NAME")):
    ax.plot(well_data['DEPTH'], well_data['BVO'], label='SWT')
    ax.plot(well_data['DEPTH'], well_data['BVOH'], label='SHF')
    # ax.set_ytitle(well)
    ax.set_ylabel(well)
    ax.set_ylim(0, .5)
    ax.legend()

plt.xlabel('DEPTH')
plt.show()

In [ ]:
import pandas as pd
from sklearn.metrics import (r2_score, mean_absolute_percentage_error, mean_absolute_error,
                             mean_squared_error, root_mean_squared_error)

vars_dict = {
    'PHIT': 'CPORE',
    'PERM': 'CPERM',
    'SWT': 'SW',
    'BVO': 'BVOH'
}

for var, compare_var in vars_dict.items():
    data = df[[var, compare_var]].dropna()
    r2 = r2_score(data[compare_var], data[var])
    mape = mean_absolute_percentage_error(data[compare_var], data[var])
    mae = mean_absolute_error(data[compare_var], data[var])
    rmse = root_mean_squared_error(data[compare_var], data[var])
    mse = mean_squared_error(data[compare_var], data[var])
    print(f"{var} R2: {r2:.2f}, MAPE: {mape:.2f}, MAE: {mae:.2f}, RMSE: {rmse:.2f}, MSE: {mse:.2f}")

results = []
for well_name, well_data in df.groupby('WELL_NAME'):
    well_results = {'WELL_NAME': well_name}
    for var, compare_var in vars_dict.items():
        data = well_data[[var, compare_var]].dropna()
        if data.empty:
            continue
        r2 = r2_score(data[compare_var], data[var])
        mape = mean_absolute_percentage_error(data[compare_var], data[var])
        mae = mean_absolute_error(data[compare_var], data[var])
        rmse = root_mean_squared_error(data[compare_var], data[var])
        mse = mean_squared_error(data[compare_var], data[var])
        well_results[f"{var}_R2"] = r2
        well_results[f"{var}_MAPE"] = mape
        well_results[f"{var}_MAE"] = mae
        well_results[f"{var}_RMSE"] = rmse
        well_results[f"{var}_MSE"] = mse
    results.append(well_results)

results_df = pd.DataFrame(results).sort_values('PHIT_MAE')
results_df.set_index('WELL_NAME').T

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

colors = plt.cm.tab20(np.linspace(0, 1, len(df['WELL_NAME'].unique())))

for i, (well_name, well_data) in enumerate(df.groupby('WELL_NAME')):
    plt.scatter(well_data['CPORE'], well_data['PHIT'], marker='.', label=well_name, color=colors[i])
plt.title('Core vs Estimated Porosity')
plt.xlabel('Core Porosity')
plt.ylabel('Estimated Porosity')
plt.xlim(0, 0.4)
plt.ylim(0, 0.4)
plt.legend(bbox_to_anchor=(1.04, 1), loc="upper left")

In [ ]:
import matplotlib.pyplot as plt

colors = plt.cm.tab20(np.linspace(0, 1, len(df['WELL_NAME'].unique())))

for i, (well_name, well_data) in enumerate(df.groupby('WELL_NAME')):
    plt.scatter(well_data['CPERM'], well_data['PERM'], marker='.', label=well_name, color=colors[i])
plt.title('Core vs Estimated Permeability')
plt.xlabel('Core Permeability')
plt.ylabel('Estimated Permeability')
plt.xscale('log')
plt.yscale('log')
plt.xlim(1e-3, 1e4)
plt.ylim(1e-3, 1e4)
plt.legend(bbox_to_anchor=(1.04, 1), loc="upper left")

In [ ]:
sorting_dict = { 
    'HW-10': 3,
    'HW-24': 12,
    'HW-25': 13,
    'HW-26': 15,
    'HW-27': 2,
    'HW-28': 5,
    'HW-29': 6,
    'HW-3': 11,
    'HW-30': 4,
    'HW-31': 8,
    'HW-32': 9,
    'HW-4': 16,
    'HW-5': 17,
    'HW-6': 14,
    'HW-7': 10,
    'HW-8': 7,
    'HW-9': 1
}
compare_df = compare_df[compare_df['FLAG'] == 'all'].copy()
compare_df['SORT'] = compare_df['WELL_NAME'].map(sorting_dict)
compare_df.sort_values('SORT', inplace=True)

In [ ]:
compare_df.plot(kind='bar', x='WELL_NAME', y='AV_BVO', figsize=(10, 3), legend=False, title='Average BVO')

In [ ]:
compare_df.plot(kind='bar', x='WELL_NAME', y='AV_SWT', figsize=(10, 3), legend=False, title='Average SWT')

In [ ]:
results_df['SORT'] = results_df['WELL_NAME'].map(sorting_dict)
results_df.sort_values('SORT', inplace=True)

metric = ['R2', 'MAE', 'RMSE', 'MSE']
for m in metric:
    results_df.plot(kind='bar', x='WELL_NAME', y=[f'SWT_{m}'], figsize=(10, 3), title=f'SWT {m} by well', legend=False)
    if m == 'RMSE':
        print(results_df[['WELL_NAME', f'SWT_{m}']])

In [ ]:
interested_wells = ['HW-30', 'HW-29', 'HW-32', 'HW-24', 'HW-25', 'HW-6', 'HW-26', 'HW-10']
nrows = len(interested_wells)
fig, ax = plt.subplots(figsize=(15, nrows * 1.5), ncols=1, nrows=nrows)
i = 0
for well, well_data in df.groupby('WELL_NAME'):
    temp_df = well_data.copy()
    temp_df = temp_df[['DEPTH', 'SWT', 'SW']].dropna()
    if temp_df.empty or well not in interested_wells:
        continue
    rmse = root_mean_squared_error(temp_df['SW'], temp_df['SWT'])
    print(f"{well} - RMSE: {rmse:.2f}")
    ax[i].set_title(f'{well} - RMSE: {rmse:.2f}')
    ax[i].plot(temp_df.DEPTH, temp_df['SW'], label='SW Gomes')
    ax[i].plot(temp_df.DEPTH, temp_df['SWT'], label='SWT')
    ax[i].set_ylim(0, 1)
    ax[i].set_ylabel('SWT')
    ax[i].legend(bbox_to_anchor=(1.04, 1), loc="upper left")
    i += 1

ax[i - 1].set_xlabel('DEPTH')
plt.tight_layout()

In [ ]:
results_df['SORT'] = results_df['WELL_NAME'].map(sorting_dict)
results_df.sort_values('SORT', inplace=True)

metric = ['R2', 'MAE', 'RMSE', 'MSE']
for m in metric:
    results_df.plot(kind='bar', x='WELL_NAME', y=[f'BVO_{m}'], figsize=(10, 3), title=f'BVO {m} by well', legend=False)
    if m == 'RMSE':
        print(results_df[['WELL_NAME', f'BVO_{m}']])

In [ ]:
compare_df.plot(kind='bar', x='WELL_NAME', y='AV_PHIT', figsize=(10, 3), legend=False, title='Average PHIT')

In [ ]:
results_df['SORT'] = results_df['WELL_NAME'].map(sorting_dict)
results_df.sort_values('SORT', inplace=True)

metric = ['R2', 'MAE', 'RMSE', 'MSE']
for m in metric:
    results_df.plot(kind='bar', x='WELL_NAME', y=[f'PHIT_{m}'], figsize=(10, 3), title=f'PHIT {m} by well', legend=False)
    if m == 'RMSE':
        print(results_df[['WELL_NAME', f'PHIT_{m}']])

In [ ]:
compare_df.plot(kind='bar', x='WELL_NAME', y='AV_PERM_AM', figsize=(10, 3), legend=False, title='Arithmetic Mean PERM',
                logy=True, ylim=(1e-2, 1e2))
compare_df.plot(kind='bar', x='WELL_NAME', y='AV_PERM_GM', figsize=(10, 3), legend=False, title='Geometric Mean PERM',
                logy=True, ylim=(1e-2, 1e2))
compare_df.plot(kind='bar', x='WELL_NAME', y='AV_PERM_HM', figsize=(10, 3), legend=False, title='Harmonic Mean PERM',
                logy=True, ylim=(1e-2, 1e2))

In [ ]:
metric = ['R2', 'MAE', 'RMSE', 'MSE']
for m in metric:
    results_df.plot(kind='bar', x='WELL_NAME', y=[f'PERM_{m}'], figsize=(10, 3), title=f'PERM {m} by well', legend=False)
    if m == 'RMSE':
        print(results_df[['WELL_NAME', f'PERM_{m}']])
print('Average RMSE:', results_df['PERM_RMSE'].mean())

In [ ]:
from quick_pp.porosity import dt_matrix, sonic_porosity_hunt_raymer, sonic_porosity_wyllie

import matplotlib.pyplot as plt

hw = df[df.WELL_NAME == 'HW-3']

dt_ma = dt_matrix(vcalc=np.ones(hw.shape[0]))
son_por = sonic_porosity_hunt_raymer(hw.DT, dt_matrix=dt_ma, dt_fluid=190)
son_por_w = sonic_porosity_wyllie(hw.DT, dt_matrix=dt_ma, dt_fluid=190)

fig, axs = plt.subplots(3, 1, figsize=(20, 6), sharex=True)
axs[0].plot(hw.DEPTH, hw.PHIT, label='PHIT')
axs[0].plot(hw.DEPTH, son_por, label='PHIS_HR')
axs[0].plot(hw.DEPTH, son_por_w, label='PHIS_W')
axs[0].scatter(hw.DEPTH, hw.CPORE, label='CPORE', marker='.', c='red')
axs[0].set_title('HW-3')
axs[0].set_ylabel('Porosity (v/v)')
axs[0].set_ylim(0, 0.5)
axs[0].legend(bbox_to_anchor=(1.01, 1), loc='upper left')

axs[1].plot(hw.DEPTH, hw.RHOB, label='RHOB')
axs[1].set_ylabel('Bulk Density (g/cc)')
axs[1].set_ylim(1.95, 2.95)
axs[1].legend(bbox_to_anchor=(1.01, 1), loc='upper left')

axs[2].plot(hw.DEPTH, hw.DT, label='DT')
axs[2].plot(hw.DEPTH, dt_ma, label='DT_MA')
axs[2].set_ylabel('Time Travel (ms/ft)')
axs[2].set_ylim(20, 100)
axs[2].legend(bbox_to_anchor=(1.01, 1), loc='upper left')
axs[2].set_xlabel('Depth (ft)')



In [ ]:
from quick_pp.porosity import dt_matrix, sonic_porosity_hunt_raymer, sonic_porosity_wyllie

import matplotlib.pyplot as plt

name = 'HW-5'
hw = df[df.WELL_NAME == name]

dt_ma = dt_matrix(vcalc=np.ones(hw.shape[0]))
son_por = sonic_porosity_hunt_raymer(hw.DT, dt_matrix=dt_ma, dt_fluid=190)
son_por_w = sonic_porosity_wyllie(hw.DT, dt_matrix=dt_ma, dt_fluid=190)

fig, axs = plt.subplots(3, 1, figsize=(20, 6), sharex=True)
axs[0].plot(hw.DEPTH, hw.PHIT, label='PHIT')
axs[0].plot(hw.DEPTH, son_por, label='PHIS_HR')
axs[0].plot(hw.DEPTH, son_por_w, label='PHIS_W')
axs[0].scatter(hw.DEPTH, hw.CPORE, label='CPORE', marker='.', c='red')
axs[0].set_title(name)
axs[0].set_ylabel('Porosity (v/v)')
axs[0].set_ylim(0, 0.5)
axs[0].legend(bbox_to_anchor=(1.01, 1), loc='upper left')

axs[1].plot(hw.DEPTH, hw.RHOB, label='RHOB')
axs[1].set_ylabel('Bulk Density (g/cc)')
axs[1].set_ylim(1.95, 2.95)
axs[1].legend(bbox_to_anchor=(1.01, 1), loc='upper left')

axs[2].plot(hw.DEPTH, hw.DT, label='DT')
axs[2].plot(hw.DEPTH, dt_ma, label='DT_MA')
axs[2].set_ylabel('Time Travel (ms/ft)')
axs[2].set_ylim(20, 100)
axs[2].legend(bbox_to_anchor=(1.01, 1), loc='upper left')
axs[2].set_xlabel('Depth (ft)')



In [ ]:
from quick_pp.porosity import dt_matrix, sonic_porosity_hunt_raymer, sonic_porosity_wyllie

import matplotlib.pyplot as plt

well_name = 'HW-27'
hw = df[df.WELL_NAME == well_name]

dt_ma = dt_matrix(vcalc=np.ones(hw.shape[0]))
son_por = sonic_porosity_hunt_raymer(hw.DT, dt_matrix=dt_ma, dt_fluid=190)
son_por_w = sonic_porosity_wyllie(hw.DT, dt_matrix=dt_ma, dt_fluid=190)

fig, axs = plt.subplots(3, 1, figsize=(20, 6), sharex=True)
axs[0].plot(hw.DEPTH, hw.PHIT, label='PHIT')
# axs[0].plot(hw.DEPTH, hw.PHID, label='PHID')
axs[0].plot(hw.DEPTH, son_por, label='PHIS_HR')
axs[0].plot(hw.DEPTH, son_por_w, label='PHIS_W')
axs[0].scatter(hw.DEPTH, hw.CPORE, label='CPORE', marker='.', c='red')
axs[0].set_title(well_name)
axs[0].set_ylabel('Porosity (v/v)')
axs[0].set_ylim(0, 0.5)
axs[0].legend(bbox_to_anchor=(1.01, 1), loc='upper left')

axs[1].plot(hw.DEPTH, hw.RHOB, label='RHOB')
axs[1].set_ylabel('Bulk Density (g/cc)')
axs[1].set_ylim(1.95, 2.95)
axs[1].legend(bbox_to_anchor=(1.01, 1), loc='upper left')

axs[2].plot(hw.DEPTH, hw.DT, label='DT')
axs[2].plot(hw.DEPTH, dt_ma, label='DT_MA')
axs[2].set_ylabel('Time Travel (ms/ft)')
axs[2].set_ylim(20, 100)
axs[2].legend(bbox_to_anchor=(1.01, 1), loc='upper left')
axs[2].set_xlabel('Depth (ft)')



In [ ]:
from sklearn.preprocessing import MinMaxScaler
from quick_pp.rock_type import calc_fzi_perm
from quick_pp.core_calibration import perm_transform

import matplotlib.pyplot as plt

well_name = 'HW-30'
hw = df[df.WELL_NAME == well_name].copy()

hw['LOG_RT'] = np.log10(hw['RT'])
hw['NDI'] = (2.95 - hw['RHOB']) / 1.95
hw['GRN'] = hw.groupby('WELL_NAME')['GR'].transform(
    lambda x: MinMaxScaler().fit_transform(x.values.reshape(-1, 1)).flatten())

input_features = ['NPHI', 'NDI', 'LOG_RT', 'GRN']

poroperm_params = {
    1.0: (192699, 3.197),
    2.0: (475543, 4.231),
    3.0: (69710, 3.503),
    4.0: (36335, 3.528),
    5.0: (25862, 3.832),
    6.0: (6336, 3.35),
    7.0: (4120, 3.752),
    8.0: (2207, 3.921),
    9.0: (801, 3.77),
    10.0: (730, 4.13),
    11.0: (335, 4.003),
    12.0: (222, 4.32)
}

# Permeability prediction based on Rock Type ML model
input_features = ['GR', 'NPHI', 'RHOB', 'NDI', 'RT', 'LOG_RT', 'GRN']
rock_flag_ml = fzi_rt_model.predict(hw[input_features])
perm_a_ml = pd.Series(rock_flag_ml).map(poroperm_params).apply(lambda x: x[0] if type(x) == tuple else np.nan)
perm_b_ml = pd.Series(rock_flag_ml).map(poroperm_params).apply(lambda x: x[1] if type(x) == tuple else np.nan)
perm_rt_ml_transform = perm_transform(hw['PHIT'], perm_a_ml, perm_b_ml)


# Permeability prediction based on FZI ML model followed by back calculate from FZI
temp_df = hw.copy()
temp_df['ROCK_PRED'] = rock_flag_ml
input_features = ['GR', 'NPHI', 'RHOB', 'NDI', 'RT', 'LOG_RT', 'GRN', 'ROCK_PRED']
temp_df[input_features] = imp_mean.fit_transform(temp_df[input_features])
fzi_ml = 10**(fzi_model.predict(temp_df[input_features]))
perm_fzi_ml = calc_fzi_perm(fzi_ml, hw['PHIT'])

# Permeability prediction based on FZI ML model, rock type and perm transform
fzi_cutoffs = [
    0.143, 0.241, 0.332, 0.469, 0.733, 1.117, 1.866, 2.506, 3.846, 5.26, 7.43
]
rock_flag = rock_typing(fzi_ml, fzi_cutoffs, higher_is_better=True)
perm_a = pd.Series(rock_flag).map(poroperm_params).apply(lambda x: x[0] if type(x) == tuple else np.nan)
perm_b = pd.Series(rock_flag).map(poroperm_params).apply(lambda x: x[1] if type(x) == tuple else np.nan)
perm_fzi_ml_transform = perm_transform(hw['PHIT'], perm_a, perm_b)

fig, axs = plt.subplots(2, 1, figsize=(20, 4), sharex=True)
axs[0].plot(hw.DEPTH, perm_rt_ml_transform, label='PERM Transform - Rock Type ML')
axs[0].plot(hw.DEPTH, perm_fzi_ml, label='PERM - FZI ML')
axs[0].plot(hw.DEPTH, hw.PERM, label='PERM - FZI ML')
axs[0].plot(hw.DEPTH, perm_fzi_ml_transform, label='PERM Transform - FZI ML')
axs[0].scatter(hw.DEPTH, hw.CPERM, label='CPERM', marker='.', c='red')
axs[0].set_title(well_name)
axs[0].set_ylabel('Permeability (mD)')
axs[0].set_yscale('log')
axs[0].set_ylim(1e-3, 1e4)
axs[0].legend(bbox_to_anchor=(1, 1), loc='upper left')

axs[1].plot(hw.DEPTH, rock_flag_ml, label='Rock Type ML')
axs[1].plot(hw.DEPTH, rock_flag, label='Rock Type FZI')
axs[1].legend(bbox_to_anchor=(1, 1), loc='upper left')
axs[1].set_xlabel('Depth (ft)')
axs[1].set_yticks([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12])
axs[1].set_ylabel('Rock Flag')

In [ ]:
err_df = hw.copy()
err_df = err_df.dropna(subset=['PERM', 'CPERM'])
err_df['error'] = err_df.CPERM - err_df.PERM
err_df[err_df.error.abs() > 50][['DEPTH', 'PERM', 'CPERM', 'error']].round(1)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from quick_pp.rock_type import calc_fzi_perm
from quick_pp.core_calibration import perm_transform

import matplotlib.pyplot as plt

well_name = 'HW-32'
hw = df[df.WELL_NAME == well_name].copy()

hw['LOG_RT'] = np.log10(hw['RT'])
hw['NDI'] = (2.95 - hw['RHOB']) / 1.95
hw['GRN'] = hw.groupby('WELL_NAME')['GR'].transform(
    lambda x: MinMaxScaler().fit_transform(x.values.reshape(-1, 1)).flatten())

input_features = ['NPHI', 'NDI', 'LOG_RT', 'GRN']

# Permeability prediction based on Rock Type ML model
input_features = ['GR', 'NPHI', 'RHOB', 'NDI', 'RT', 'LOG_RT', 'GRN']
rock_flag_ml = fzi_rt_model.predict(hw[input_features])
perm_a_ml = pd.Series(rock_flag_ml).map(poroperm_params).apply(lambda x: x[0] if type(x) == tuple else np.nan)
perm_b_ml = pd.Series(rock_flag_ml).map(poroperm_params).apply(lambda x: x[1] if type(x) == tuple else np.nan)
perm_rt_ml_transform = perm_transform(hw['PHIT'], perm_a_ml, perm_b_ml)


# Permeability prediction based on FZI ML model followed by back calculate from FZI
temp_df = hw.copy()
temp_df['ROCK_PRED'] = rock_flag_ml
input_features = ['GR', 'NPHI', 'RHOB', 'NDI', 'RT', 'LOG_RT', 'GRN', 'ROCK_PRED']
temp_df[input_features] = imp_mean.fit_transform(temp_df[input_features])
fzi_ml = 10**(fzi_model.predict(temp_df[input_features]))
perm_fzi_ml = calc_fzi_perm(fzi_ml, hw['PHIT'])

# Permeability prediction based on FZI ML model, rock type and perm transform
rock_flag = rock_typing(fzi_ml, fzi_cutoffs, higher_is_better=True)
perm_a = pd.Series(rock_flag).map(poroperm_params).apply(lambda x: x[0] if type(x) == tuple else np.nan)
perm_b = pd.Series(rock_flag).map(poroperm_params).apply(lambda x: x[1] if type(x) == tuple else np.nan)
perm_fzi_ml_transform = perm_transform(hw['PHIT'], perm_a, perm_b)

fig, axs = plt.subplots(2, 1, figsize=(20, 4), sharex=True)
axs[0].plot(hw.DEPTH, perm_rt_ml_transform, label='PERM Rock Flag ML Transform')
axs[0].plot(hw.DEPTH, perm_fzi_ml, label='PERM FZI ML')
axs[0].plot(hw.DEPTH, perm_fzi_ml_transform, label='PERM FZI ML Transform')
axs[0].scatter(hw.DEPTH, hw.CPERM, label='CPERM', marker='.', c='red')
axs[0].set_title(well_name)
axs[0].set_ylabel('Permeability (mD)')
axs[0].set_yscale('log')
axs[0].set_ylim(1e-3, 1e3)
axs[0].legend(bbox_to_anchor=(1.01, 1), loc='upper left')

axs[1].plot(hw.DEPTH, rock_flag_ml, label='Rock Flag ML')
axs[1].plot(hw.DEPTH, rock_flag, label='Rock Type FZI')
axs[1].legend(bbox_to_anchor=(1.01, 1), loc='upper left')
axs[1].set_xlabel('Depth (ft)')
axs[1].set_yticks([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12])
axs[1].set_ylabel('Rock Flag')

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from quick_pp.rock_type import calc_fzi_perm
from quick_pp.core_calibration import perm_transform
from quick_pp.porosity import dt_matrix, sonic_porosity_hunt_raymer

import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator

well_name = 'HW-27'
hw = df[df.WELL_NAME == well_name].copy()

dt_ma = dt_matrix(vcalc=np.ones(hw.shape[0]))
son_por = sonic_porosity_hunt_raymer(hw.DT, dt_matrix=dt_ma, dt_fluid=190)

hw['LOG_RT'] = np.log10(hw['RT'])
hw['NDI'] = (2.95 - hw['RHOB']) / 1.95
hw['GRN'] = hw.groupby('WELL_NAME')['GR'].transform(
    lambda x: MinMaxScaler().fit_transform(x.values.reshape(-1, 1)).flatten())

input_features = ['NPHI', 'NDI', 'LOG_RT', 'GRN']

# Permeability prediction based on Rock Type ML model
input_features = ['GR', 'NPHI', 'RHOB', 'NDI', 'RT', 'LOG_RT', 'GRN']
rock_flag_ml = fzi_rt_model.predict(hw[input_features])
perm_a_ml = pd.Series(rock_flag_ml).map(poroperm_params).apply(lambda x: x[0] if type(x) == tuple else np.nan)
perm_b_ml = pd.Series(rock_flag_ml).map(poroperm_params).apply(lambda x: x[1] if type(x) == tuple else np.nan)
perm_rt_ml_transform = perm_transform(hw['PHIT'], perm_a_ml, perm_b_ml)


# Permeability prediction based on FZI ML model followed by back calculate from FZI
temp_df = hw.copy()
temp_df['ROCK_PRED'] = rock_flag_ml
input_features = ['GR', 'NPHI', 'RHOB', 'NDI', 'RT', 'LOG_RT', 'GRN', 'ROCK_PRED']
temp_df[input_features] = imp_mean.fit_transform(temp_df[input_features])
fzi_ml = 10**(fzi_model.predict(temp_df[input_features]))
perm_fzi_ml = calc_fzi_perm(fzi_ml, hw['PHIT'])

# Permeability prediction based on Rock Type ML using sonic porosity
perm_rt_ml_sonic = perm_transform(son_por, perm_a_ml, perm_b_ml)

# Permeability prediction based on FZI ML model, rock type and perm transform using sonic porosity
perm_fzi_ml_sonic = calc_fzi_perm(fzi_ml, son_por)

# Permeability prediction based on FZI ML model, rock type and perm transform
rock_flag = rock_typing(fzi_ml, fzi_cutoffs, higher_is_better=True)
perm_a = pd.Series(rock_flag).map(poroperm_params).apply(lambda x: x[0] if type(x) == tuple else np.nan)
perm_b = pd.Series(rock_flag).map(poroperm_params).apply(lambda x: x[1] if type(x) == tuple else np.nan)
perm_fzi_ml_transform = perm_transform(hw['PHIT'], perm_a, perm_b)

fig, axs = plt.subplots(2, 1, figsize=(20, 4), sharex=True)
axs[0].plot(hw.DEPTH, perm_rt_ml_transform, label='PERM Transform - Rock Flag ML')
axs[0].plot(hw.DEPTH, perm_fzi_ml, label='PERM - FZI ML')
axs[0].plot(hw.DEPTH, perm_fzi_ml_transform, label='PERM Transform - FZI ML')
axs[0].plot(hw.DEPTH, perm_fzi_ml_sonic, label='PERM - FZI ML Sonic Porosity')
axs[0].plot(hw.DEPTH, perm_rt_ml_sonic, label='PERM Transform - Sonic Porosity')
axs[0].scatter(hw.DEPTH, hw.CPERM, label='CPERM', marker='.', c='red')
axs[0].set_title(well_name)
axs[0].set_ylabel('Permeability (mD)')
axs[0].set_yscale('log')
axs[0].set_ylim(1e-3, 1e3)
axs[0].legend(bbox_to_anchor=(1.01, 1), loc='upper left')

axs[1].plot(hw.DEPTH, rock_flag_ml, label='Rock Type ML')
axs[1].plot(hw.DEPTH, rock_flag, label='Rock Type FZI')
axs[1].legend(bbox_to_anchor=(1, 1), loc='upper left')
axs[1].set_xlabel('Depth (ft)')
axs[1].set_yticks([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12])
axs[1].set_ylabel('Rock Flag')

In [ ]:

well_name = 'HW-27'
hw = df[df.WELL_NAME == well_name]
hw.columns

In [ ]:
from quick_pp.porosity import dt_matrix, sonic_porosity_hunt_raymer, sonic_porosity_wyllie

import matplotlib.pyplot as plt

well_name = 'HW-32'
hw = df[df.WELL_NAME == well_name].copy()
hw['BVO'] = hw['PHIT'] * (1 - hw['SWT'])
hw['BVOH'] = hw['PHIT'] * (1 - hw['SHF'])

fig, axs = plt.subplots(3, 1, figsize=(20, 6), sharex=True)
axs[0].set_title(well_name)
axs[0].plot(hw.DEPTH, hw['BVO'], label=r'$BVO_{SWT}$')
axs[0].plot(hw.DEPTH, hw['BVOH'], label=r'$BVO_{SHF}$')
axs[0].set_ylabel('BVO (v/v)')
axs[0].set_ylim(0, 0.5)
axs[0].legend(bbox_to_anchor=(1.01, 1), loc='upper left')

axs[1].plot(hw.DEPTH, hw.SWT, label='SWT')
axs[1].plot(hw.DEPTH, hw.SHF, label='SHF')
axs[1].set_ylabel('Water Saturation (v/v)')
axs[1].set_ylim(0, 1)
axs[1].legend(bbox_to_anchor=(1.01, 1), loc='upper left')

axs[2].plot(hw.DEPTH, hw.PHIT, label='PHIT')
axs[2].set_ylabel('Porosity (v/v)')
axs[2].set_ylim(0, .5)
axs[2].legend(bbox_to_anchor=(1.01, 1), loc='upper left')
axs[2].set_xlabel('Depth (ft)')



In [ ]:
from quick_pp.porosity import dt_matrix, sonic_porosity_hunt_raymer, sonic_porosity_wyllie

import matplotlib.pyplot as plt

well_name = 'HW-29'
hw = df[df.WELL_NAME == well_name].copy()
hw['BVO'] = hw['PHIT'] * (1 - hw['SWT'])
hw['BVOH'] = hw['PHIT'] * (1 - hw['SHF'])

fig, axs = plt.subplots(3, 1, figsize=(20, 6), sharex=True)
axs[0].set_title(well_name)
axs[0].plot(hw.DEPTH, hw['BVO'], label=r'$BVO_{SWT}$')
axs[0].plot(hw.DEPTH, hw['BVOH'], label=r'$BVO_{SHF}$')
axs[0].set_ylabel('BVO (v/v)')
axs[0].set_ylim(-.1, .4)
axs[0].legend(bbox_to_anchor=(1.01, 1), loc='upper left')

axs[1].plot(hw.DEPTH, hw.SWT, label='SWT')
axs[1].plot(hw.DEPTH, hw.SHF, label='SHF')
axs[1].set_ylabel('Water Saturation (v/v)')
axs[1].set_ylim(0, 1)
axs[1].legend(bbox_to_anchor=(1.01, 1), loc='upper left')

axs[2].plot(hw.DEPTH, hw.PHIT, label='PHIT')
axs[2].set_ylabel('Porosity (v/v)')
axs[2].set_ylim(-.1, .4)
axs[2].legend(bbox_to_anchor=(1.01, 1), loc='upper left')
axs[2].set_xlabel('Depth (ft)')



In [ ]:
from quick_pp.porosity import dt_matrix, sonic_porosity_hunt_raymer, sonic_porosity_wyllie

import matplotlib.pyplot as plt

well_name = 'HW-30'
hw = df[df.WELL_NAME == well_name].copy()
hw['BVO'] = hw['PHIT'] * (1 - hw['SWT'])
hw['BVOH'] = hw['PHIT'] * (1 - hw['SHF'])

fig, axs = plt.subplots(3, 1, figsize=(20, 6), sharex=True)
axs[0].set_title(well_name)
axs[0].plot(hw.DEPTH, hw['BVO'], label=r'$BVO_{SWT}$')
axs[0].plot(hw.DEPTH, hw['BVOH'], label=r'$BVO_{SHF}$')
axs[0].set_ylabel('BVO (v/v)')
axs[0].set_ylim(0, 0.5)
axs[0].legend(bbox_to_anchor=(1.01, 1), loc='upper left')

axs[1].plot(hw.DEPTH, hw.SWT, label='SWT')
axs[1].plot(hw.DEPTH, hw.SHF, label='SHF')
axs[1].set_ylabel('Water Saturation (v/v)')
axs[1].set_ylim(0, 1)
axs[1].legend(bbox_to_anchor=(1.01, 1), loc='upper left')

axs[2].plot(hw.DEPTH, hw.PHIT, label='PHIT')
axs[2].set_ylabel('Porosity (v/v)')
axs[2].set_ylim(0, .5)
axs[2].legend(bbox_to_anchor=(1.01, 1), loc='upper left')
axs[2].set_xlabel('Depth (ft)')



In [ ]:
from quick_pp.porosity import dt_matrix, sonic_porosity_hunt_raymer, sonic_porosity_wyllie

import matplotlib.pyplot as plt

well_name = 'HW-26'
hw = df[df.WELL_NAME == well_name].copy()
hw['BVO'] = hw['PHIT'] * (1 - hw['SWT'])
hw['BVOH'] = hw['PHIT'] * (1 - hw['SHF'])

fig, axs = plt.subplots(3, 1, figsize=(20, 6), sharex=True)
axs[0].set_title(well_name)
axs[0].plot(hw.DEPTH, hw['BVO'], label=r'$BVO_{SWT}$')
axs[0].plot(hw.DEPTH, hw['BVOH'], label=r'$BVO_{SHF}$')
axs[0].set_ylabel('BVO (v/v)')
axs[0].set_ylim(0, 0.5)
axs[0].legend(bbox_to_anchor=(1.01, 1), loc='upper left')

axs[1].plot(hw.DEPTH, hw.SWT, label='SWT')
axs[1].plot(hw.DEPTH, hw.SHF, label='SHF')
axs[1].set_ylabel('Water Saturation (v/v)')
axs[1].set_ylim(0, 1)
axs[1].legend(bbox_to_anchor=(1.01, 1), loc='upper left')

axs[2].plot(hw.DEPTH, hw.PHIT, label='PHIT')
axs[2].set_ylabel('Porosity (v/v)')
axs[2].set_ylim(0, .5)
axs[2].legend(bbox_to_anchor=(1.01, 1), loc='upper left')
axs[2].set_xlabel('Depth (ft)')

